# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Цель исследования — проверить гипотезы:

*  Есть ли зависимость между количеством детей и возвратом кредита в срок?
*  Есть ли зависимость между семейным положением и возвратом кредита в срок? 
*  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
*  Как разные цели кредита влияют на его возврат в срок?

Ход исследования

Данные по статистике о платёжеспособности клиентов получаем из файла /datasets/data.csv. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.
Затем потребуется проверить данные на ошибки и оценить их влияние на исследование. Затем, на этапе предобработки исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:

* Обзор данных.
* Предобработка данных.
* Проверка гипотез.

### Шаг 1. Обзор данных

Прочитаем файл data.csv и сохраним его в переменной df. Получим первых 10 строк таблицы.

In [1]:
# импорт библиотеки pandas
import pandas as pd 
# чтение файла с данными с сохранением в df
df=pd.read_csv('/datasets/data.csv') 
# получение первых 10 строк таблицы df
df.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы df.

In [2]:
# получение общей информации о данных в таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов.

Подробно разберём, какие в df столбцы и какую информацию они содержат:

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;  
* `debt` — название жанра;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита;
  
Количество значений в столбцах различается. Это говорит о том, что в данных есть <пустые> значения.

Выводы

Каждая строка таблицы содержит информацию о клиентах банка. На первый вгляд бросаются следующие проблемы, которые нужно решать: пропуски, отрицательные значения, разный регистр, числа с плавающей точкой. Для проверки рабочих гипотез особенно ценны столбцы children, family_status, total_income и purpose. 

### Шаг 2.1 Заполнение пропусков



In [3]:
# суммарное количество пропусков, выявленных методом isna() в таблице df
df.isna().sum()   

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах days_employed и total_income есть пропущенные значения, проверим какую долю они составляют. 

In [4]:
# поделим количество пропусков на общее количество значений в столбце
print(df['days_employed'].isna().sum() / df['days_employed'].count() * 100)


11.234561521368406


Доля пропущенные значений в столбцах равна 11,2%, процент не критичный, но мы всё равно заполним пропуски.
Пропущенным значением в наших столбцах является NaN — общепринятый способ заполнять ячейку, для которых не хватает данных.


In [5]:
# посмотрим max и min значения в столбцах с пропусками
print('days_employed max:', df['days_employed'].max()) 
print('days_employed min:', df['days_employed'].min()) 
print('total_income max:', df['total_income'].max()) 
print('total_income min:', df['total_income'].min()) 

days_employed max: 401755.40047533
days_employed min: -18388.949900568383
total_income max: 2265604.028722744
total_income min: 20667.26379327158


В нашем случае есть данные оторванные от основной массы, они будут сильно смещать значение среднего вверх. Поэтому оценивать широкий диапазон лучше медианой.

In [6]:
# заполнение пропусков медианным значением
df = df.fillna(df.median())
# подсчёт пропусков 
df.isna().sum() 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пропущенные значения заполнены, переходим к следующему этапу.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Первое, что попадется в глаза - отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработаем эти значения. 

In [7]:
# меняем отрицательные значения столбца days_employed на положительные
df.loc[df['days_employed'] < 0, 'days_employed'] = df.loc[df['days_employed'] <0, 'days_employed'].abs()

In [8]:
# проверим результат
df['days_employed'].head()

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64

In [9]:
# посмотрим уникальные значения столбца 'children'
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Обнаружено отрицательное значение столбца с количеством детей, предположим, что это ошибка знака, как и у столбца 'days_employed' и изменим отрицательное значение на положительное

In [10]:
# посчитаем количество строки, значение столбца которых равно -1
df[df['children'] == -1]['children'].count()

47

In [11]:
# меняем отрицательные значения столбца children на положительные
df.loc[df['children'] < 0, 'children'] = df.loc[df['children'] <0, 'children'].abs()

In [12]:
# проверим результат
df['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

Так же в этом столбце есть аномально большое значение '20', посмотрим действительно ли это просто исключение

In [13]:
# посчитаем количество строки, значение столбца которых равно 20
df[df['children'] == 20]['children'].count()

76

Можно сделать вывод, что это ошибка в данных и количество детей на самом деле равно 2, поэтому заменим все значения с 20 на 2

In [14]:
# меняем значения равные 20 столбца children на 2
df.loc[df['children'] == 20, 'children'] = 2

In [15]:
# проверим результат
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

Аномалии в столбце 'children' исправлены

Проверим столбец dob_years на аномалии

In [16]:
# посмотрим уникальные значения столбца 'dob_years'
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Нулевое значение в столбце с возрастом, это явно аномалия

In [17]:
# посчитаем количество значений равных нулю
df[df['dob_years'] == 0]['dob_years'].count()

101

In [18]:
# заменим все значения в столбце dob_years равные нулю, на среднее значение по столбцу
df.loc[df['dob_years'] == 0, 'dob_years'] = df['dob_years'].mean()

In [19]:
# проверим результат
df[df['dob_years'] == 0]['dob_years'].count()

0

Аномалия в столбце 'dob_years' исправлена

Проверим столбец gender на аномалии

In [20]:
# посмотрим уникальные значения столбца 'gender'
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Уберем значение 'XNA' не относящееся к полу

In [21]:
# перезапишем df без значения 'XNA'
df = df[df['gender'] != 'XNA']

In [22]:
# проверим результат
df['gender'].unique()

array(['F', 'M'], dtype=object)

Аномалия в столбце 'gender' исправлена

### Шаг 2.3. Изменение типов данных.
Заменим вещественный тип данных в столбце total_income и days_employed на целочисленный c помощью метода astype().

In [23]:
df = df.astype({'total_income': 'int64', 'days_employed': 'int64'})
# проверим изменения
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.
Сначала проверим наши данные на явные дубликаты

In [24]:
# получение суммарного количества дубликатов в таблице df
df.duplicated().sum()

54

In [25]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df=df.drop_duplicates().reset_index(drop=True)
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Также обработаем неявные дубликаты. 
В столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. 
Приведем их к одному регистру. 

In [26]:
# выведем список уникальных значений 'education'
df['education'].unique() 

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [27]:
# приведем значения столбца к одному регистру
df['education'] = df['education'].str.lower()

In [28]:
# проверим результат
df['education'].unique() 

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [29]:
# получение суммарного количества дубликатов в таблице df
df.duplicated().sum()

17

In [30]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df=df.drop_duplicates().reset_index(drop=True)
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Проверим остальные столбцы.

In [31]:
# Просмотр уникальных значений столбца 'purpose'
df['purpose'].sort_values(ascending=True).unique()    

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [32]:
# Функция для замены неявных дубликатов
def replace_wrong_purpose(wrong_purpose, correct_purpose):
    for value in wrong_purpose:
        df['purpose'] = df['purpose'].replace(value, correct_purpose)
    return df['purpose']

wrong_purpose = [
        'высшее образование','дополнительное образование', 
        'заняться высшим образованием', 'заняться образованием',
        'образование', 'получение высшего образования',
        'получение дополнительного образования', 'получение образования',
        'профильное образование'         
               ]
correct_purpose = 'получение образования'

In [33]:
# вызов функции для замены неявных дубликатов
#replace_wrong_purpose(wrong_purpose, correct_purpose) 

In [34]:
# Функция для замены неявных дубликатов
def replace_wrong_purpose(wrong_purpose, correct_purpose):
    for value in wrong_purpose:
        df['purpose'] = df['purpose'].replace(value, correct_purpose)
    return df['purpose']

wrong_purpose = [
       'на проведение свадьбы',
       'свадьба', 'сыграть свадьбу'       
               ]
correct_purpose = 'проведение свадьбы'


In [35]:
# вызов функции для замены неявных дубликатов
#replace_wrong_purpose(wrong_purpose, correct_purpose) 

In [36]:
# Функция для замены неявных дубликатов
def replace_wrong_purpose(wrong_purpose, correct_purpose):
    for value in wrong_purpose:
        df['purpose'] = df['purpose'].replace(value, correct_purpose)
    return df['purpose']

wrong_purpose = [
       'автомобили', 'автомобиль', 
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 
       'приобретение автомобиля', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем'       
               ]
correct_purpose = 'операции с автомобилем'


In [37]:
# вызов функции для замены неявных дубликатов
#replace_wrong_purpose(wrong_purpose, correct_purpose) 

In [38]:
# Функция для замены неявных дубликатов
def replace_wrong_purpose(wrong_purpose, correct_purpose):
    for value in wrong_purpose:
        df['purpose'] = df['purpose'].replace(value, correct_purpose)
    return df['purpose']
wrong_purpose = [
    'покупка жилья', 'недвижимость', 'операции с жильем', 
    'операции с коммерческой недвижимостью', 'операции с недвижимостью', 
    'операции со своей недвижимостью' 'покупка жилой недвижимости', 'покупка жилья',
    'покупка жилья для сдачи', 'покупка жилья для семьи', 'ремонт жилью',
    'покупка коммерческой недвижимости', 'покупка недвижимости',
    'покупка своего жилья', 'строительство жилой недвижимости', 
    'строительство недвижимости', 'строительство собственной недвижимости', 
    'покупка жилой недвижимости', 'жилье', 'операции со своей недвижимостью'
               ]
correct_purpose = 'операции с недвижимостью'

In [39]:
# вызов функции для замены неявных дубликатов
#replace_wrong_purpose(wrong_purpose, correct_purpose) 

In [40]:
# проверим результат
df['purpose'].unique() 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [41]:
# Просмотр уникальных значений столбца 'family_status'
df['family_status'].unique() 

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

На данные это не повлияет, но приведём все значения в столбце family_status к одному регистру

In [42]:
# приведем значения столбца к одному регистру
df['family_status'] = df['family_status'].str.lower()

In [43]:
# проверим результат
df['family_status'].unique() 

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [44]:
# Просмотр уникальных значений столбца 'income_type'
df['income_type'].unique() 

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Неявные дубликаты в столбце income_type не обнаружены

Все столбцы проверены на явные и неявные дубликаты и исправлены.

Дубликаты могли появиться вследствие заполнения данных пользователями отсюда разный регистр в столбце education, разные формулировки цели кредита в столбце purpose, 

               

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:	
* каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
* каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.


In [45]:
df_education = df[['education', 'education_id']].drop_duplicates().reset_index(drop = True)
df_education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [46]:
df_family_status = df[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop = True)
df_family_status 

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id.

In [47]:
# методом drop удалим из df соотвествующие столбцы
df = df.drop(columns = ['education', 'family_status'], axis = 1) 
# проверим результат
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:
*	0–30000 — 'E';
*	30001–50000 — 'D';
*	50001–200000 — 'C';
*	200001–1000000 — 'B';
*	1000001 и выше — 'A'.


In [48]:
# создание функции для категоризации доходов
def income_group(income):
    if income <= 30000:
        return 'E'
    if 30000 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'
# создадим столбец total_income_category с категориями дохода    
df['total_income_category'] = df['total_income'].apply(income_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
*	'операции с автомобилем',
*	'операции с недвижимостью',
*	'проведение свадьбы',
*	'получение образования'.


In [49]:
def purpose_categorizer(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    return purpose # важно добавить крайний случай иначе занулит значения которые не подошли к шаблону
# создадим столбец purpose_category с категориями целей кредита  
df['purpose_category'] = df['purpose'].apply(purpose_categorizer) 

In [50]:
df['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

### Ответы на вопросы.

Данные очищенны. Теперь можно сгрупировать их для того чтобы ответить на поставленые вопросы в задании

In [51]:
late_payment = df[(df['debt'] == 1)].count() / df[(df['debt'] == 0)].count() * 100
late_payment['debt']

8.832183441558442

в 8,8% случаем кредит не выплачивается вовремя. Посмотрим как меняется это значение в зависимости от других факторов.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [52]:
children_to_debt = df.groupby('children').agg({'debt': ['count', 'sum']})
children_to_debt['% просрочки'] = children_to_debt['debt']['sum'] / children_to_debt['debt']['count'] * 100
children_pivot = children_to_debt.pivot_table(index='children', values='% просрочки') 
children_pivot.head()

children
0    7.544358
1    9.165808
2    9.492481
3    8.181818
4    9.756098
Name: % просрочки, dtype: float64

##### Вывод 1: 

Люди у которых нет детей имеют меньше проблем с возвратом кредита (7.5% просрочек), чем люди, которые имеют одного и более детей (8,2 - 9.7% просрочек)

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [53]:
family_status_grouped = df.groupby('family_status_id').agg({'debt': ['count', 'sum']})
family_status_grouped['% просрочки'] = family_status_grouped['debt']['sum'] / family_status_grouped['debt']['count'] * 100
family_status_grouped = family_status_grouped.reset_index(drop=False)
family_status_grouped.head()

family_status_id   debt      % просрочки
                    count  sum            
0                0  12339  931    7.545182
1                1   4150  388    9.349398
2                2    959   63    6.569343
3                3   1195   85    7.112971
4                4   2810  274    9.750890

In [54]:
family_status_grouped = family_status_grouped.pivot_table(index='family_status_id', values='% просрочки')
family_status_grouped 

/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


family_status_id
0    7.545182
1    9.349398
2    6.569343
3    7.112971
4    9.750890
Name: % просрочки, dtype: float64

##### Вывод 2: 

Не женатые/ Не замужние или люди в гражданском браке реже выплачивают кредит в срок.

##### Вопрос 3:	Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [55]:
income_pivot = df.pivot_table(index='total_income_category',values='debt', aggfunc=['count', 'mean'])
income_pivot

,count,mean
,debt,debt
total_income_category,,
A,25,0.080000
B,5040,0.070635
C,16016,0.084915
D,350,0.060000
E,22,0.090909


##### Вывод 3:

Люди с наименьшим доходом чаще других групп имеют проблемы с выплатой кредита в срок

##### Вопрос 4:	Как разные цели кредита влияют на его возврат в срок?

In [56]:
purpose_pivot = df.pivot_table(index='purpose_category',values='debt', aggfunc=['count', 'mean']).reset_index(drop=False)
purpose_pivot

,purpose_category,count,mean
,,debt,debt
0,операции с автомобилем,4306,0.093590
1,операции с недвижимостью,10810,0.072340
2,получение образования,4013,0.092200
3,проведение свадьбы,2324,0.080034


##### Вывод 4:

 Люди с большей вероятностью выплачивают кредит если он взят на операции с жильем или недвижимостью или на свадьбу. 

## Общий вывод:

При работе с данными были заполнены пропущенные значения, удалены дубликаты и выделены категории для целей кредита, группы по возрасту и образованию. Применив сводные таблицы, я оценил, какие критерии влияют на возврат кредита. Дополнительный анализ может помочь более точно предсказать вероятность вылпаты кредита в срок.